In [ ]:
# 업무추진비 원본 데이터 정제 코드

# 코드 흐름
# 1. 엑셀 파일
# 2. 시트별로 헤더 자동 탐색
# 3. 컬럼 매핑 후보 정의
# 4. 통일된 구조로 통합

import pandas as pd
import os
import re

# 1. 헤더 자동 탐색
def find_header_row(df):
    for i in range(min(15, len(df))):  # 범위 확대
        row = df.iloc[i].astype(str)
        row = row.str.replace("\n", "").str.strip()
        if row.str.contains("사용자|일자|금액|장소|목적|인원").sum() >= 2:
            return i
    return None

# 2. 컬럼 매핑 후보
column_mapping_candidates = {
    '사용자': ['사용자'],
    '사용일시': [
        '사용일시', '사용일자', '사용일자(일시)', '일자', '일시',
        '사용일자\n(일시)', '사용일시\n(일시)','결제일시',
    ],
    '장소': ['장소', '장소(가맹점명)', '장소/대상', '사용장소', '사용장소(가맹점명)'],
    '집행금액': ['금액', '금액(원)', '사용금액', '사용금액(원)'],
    '사용목적': ['집행목적', '집행목적(내역)', '사용목적(내역)', ' 집행내역(목적)'],
    '인원수': ['인원','대상', '인원수(명)', '대상인원(명)', '대상인원수', '대상인원수(명)']
}

# 3. 컬럼 매핑 함수
def match_columns(columns):
    mapping = {}
    cleaned_cols = [str(col).replace("\n", "").strip() for col in columns]
    for std_col, candidates in column_mapping_candidates.items():
        for candidate in candidates:
            for i, raw_col in enumerate(columns):
                if candidate.strip() == cleaned_cols[i]:
                    mapping[std_col] = raw_col
                    break
            if std_col in mapping:
                break
    return mapping

# 4. 메인 처리 함수 (다중 시트 포함)
def process_spending_folder_with_sheets(input_folder, output_file="정제된_업무추진비.xlsx"):
    all_dataframes = []
    errors = []

    for file in os.listdir(input_folder):
        if not file.lower().endswith('.xlsx'):
            continue

        path = os.path.join(input_folder, file)
        try:
            xls = pd.ExcelFile(path)
            for sheet_name in xls.sheet_names:
                try:
                    df = xls.parse(sheet_name, header=None)
                    header_row = find_header_row(df)
                    if header_row is None:
                        raise ValueError("헤더 행을 찾을 수 없음")

                    df.columns = df.iloc[header_row].astype(str).str.replace("\n", "").str.strip()
                    df = df.iloc[header_row+1:].copy()
                    df = df.dropna(how='all')
                    df.columns.name = None

                    mapping = match_columns(df.columns)
                    selected_data = {}
                    for std_col in ['사용자', '사용일시', '장소', '집행금액', '사용목적', '인원수']:
                        if std_col in mapping:
                            selected_data[std_col] = df[mapping[std_col]]
                        else:
                            selected_data[std_col] = pd.NA

                    df_selected = pd.DataFrame(selected_data)

                    df_selected['집행금액'] = df_selected['집행금액'].replace(",", "", regex=True)
                    df_selected['집행금액'] = pd.to_numeric(df_selected['집행금액'], errors='coerce')

                    df_selected["파일명"] = file
                    df_selected["시트명"] = sheet_name
                    all_dataframes.append(df_selected)
                except Exception as sheet_error:
                    errors.append((f"{file} - [{sheet_name}]", str(sheet_error)))
        except Exception as file_error:
            errors.append((file, str(file_error)))

    # 병합 및 정렬
    result_df = pd.concat(all_dataframes, ignore_index=True)
    ordered_cols = ['사용자', '사용일시', '장소', '집행금액', '사용목적', '인원수', '파일명', '시트명']
    for col in ordered_cols:
        if col not in result_df.columns:
            result_df[col] = pd.NA
    result_df = result_df[ordered_cols]

    # 저장
    output_path = os.path.join(input_folder, output_file)
    result_df.to_excel(output_path, index=False)
    print(f"✅ 정제 완료 → {output_path}")

    if errors:
        print("\n⚠️ 오류 발생 시트 목록:")
        for name, reason in errors:
            print(f"- {name}: {reason}")

# 사용 예시
if __name__ == "__main__":
    input_folder = r"C:\bit_esg\python\downloads"  # ← 경로 수정
    process_spending_folder_with_sheets(input_folder)


In [ ]:
# 사용일시 정규화 + 기간 필터링

import pandas as pd
from dateutil.parser import parse

# 1) 엑셀 파일 불러오기
df = pd.read_excel('정제된_업무추진비.xlsx')

# 2) 원본 날짜 컬럼명이 '날짜'라고 가정.
#    실제 컬럼명에 맞게 바꿔주세요.
col = '사용일시'

# 3) 파싱 함수 정의
def normalize_date(x):
    """
    가능한 모든 날짜 포맷을 우선 dateutil에 맡겨 파싱.
    실패하면 dayfirst=True로 재시도, 그래도 실패 시 NaT 반환.
    """
    if pd.isna(x):
        return pd.NaT
    s = str(x).strip()
    for dayfirst in (False, True):
        try:
            dt = parse(s, yearfirst=True, dayfirst=dayfirst)
            # datetime.date 객체로 변환 후 YYYY-MM-DD 포맷 문자열로 리턴
            return dt.date().isoformat()
        except Exception:
            continue
    return pd.NaT

# 4) 컬럼 전체에 적용
df[col] = df[col].apply(normalize_date)

# 5) 파싱 실패(=NaT) 행 확인 (필요 시 수작업 보완)
failed = df[df[col].isna()]
if not failed.empty:
    print("아래 행의 날짜 파싱에 실패했습니다. 직접 확인하세요:")
    print(failed)

# 6) 이제 필터링(2023년 이후)도 바로 적용할 수 있습니다.
df = df[df[col] >= '2023-01-01']

# 7) 결과 저장
df.to_excel('업무추진비_정제_2023이후.xlsx', index=False)
print(f"총 {len(df)}개 행이 남았습니다.")


In [19]:
# 사용목적이 식사 관련인 행만 필터링해서 정제하는 코드

import pandas as pd

# 1. 엑셀 파일 경로 (입력 파일)
file_path = r"C:\bit_esg\python\data_anal\업무추진비_정제_2023이후.xlsx"

# 2. 엑셀 데이터 불러오기
df = pd.read_excel(file_path)

# 3. 식사 관련 키워드 정의
meal_keywords = [
    "오찬", "만찬", "중식", "석식", "식사", "회식", "간담회", "간담",
    "송년회", "신년회", "환송", "초청"
]

# 4. 사용목적에 키워드가 포함된 행만 필터링
filtered_df = df[
    df["사용목적"].notna() &
    df["사용목적"].astype(str).str.contains("|".join(meal_keywords), case=False)
].copy()

# 5. 주요 컬럼 정제
filtered_df["장소"] = filtered_df["장소"].astype(str).str.strip()
filtered_df["사용일시"] = pd.to_datetime(filtered_df["사용일시"], errors="coerce")
filtered_df["집행금액"] = pd.to_numeric(filtered_df["집행금액"], errors="coerce")

# 6. 필요한 컬럼만 유지
columns_to_keep = ["사용자", "사용일시", "장소", "집행금액", "사용목적", "인원수", "파일명", "시트명"]
filtered_df = filtered_df[columns_to_keep]

# 7. 결과 저장 경로
output_path = r"C:\bit_esg\python\data_anal\식사관련_업무추진비_정제본.xlsx"
filtered_df.to_excel(output_path, index=False)

print(f"✅ 저장 완료: {output_path}")


✅ 저장 완료: C:\bit_esg\python\data_anal\식사관련_업무추진비_정제본.xlsx


In [23]:
import pandas as pd
import re

# 1. 데이터 로드
file_path = r"C:\bit_esg\python\data_anal\식사관련_업무추진비_정제본.xlsx"
df = pd.read_excel(file_path)

# 2. 장소 문자열 정제
df["장소"] = df["장소"].astype(str).str.strip()

# 3. 배달음식 후보 키워드 패턴 정의
delivery_pattern = re.compile(r"^\(주\)|^\(유\)|배달|포장|택배|쿠팡|배민|요기요", re.IGNORECASE)

# 4. 키워드 매칭으로 1차 후보 추출
candidates = df[df["장소"].str.contains(delivery_pattern, na=False)].copy()

# 5. 후보별 금액 통계 계산
stats = candidates.groupby("장소").agg(
    거래수=("집행금액", "count"),
    평균금액=("집행금액", "mean"),
    최대금액=("집행금액", "max")
).reset_index()

# 6. 임계값(threshold) 설정 — 예: 평균금액의 75th percentile
threshold = stats["평균금액"].quantile(0.75)

# 7. 평균금액이 threshold 이상인 진짜 배달음식 후보 추출
refined = stats[stats["평균금액"] >= threshold].reset_index(drop=True)

# 8. 결과 저장 (원하시면 주석 해제)
# stats.to_excel(r"C:\bit_esg\python\data_anal\delivery_candidates_stats.xlsx", index=False)
# refined.to_excel(r"C:\bit_esg\python\data_anal\delivery_candidates_refined.xlsx", index=False)

# 9. 확인용 출력
print("=== 전체 배달음식 후보 통계 ===")
print(stats)
print(f"\n평균금액 75th percentile (threshold): {threshold:.2f}\n")
print("=== 임계값 이상으로 진짜 배달음식으로 의심되는 장소 ===")
print(refined)


=== 전체 배달음식 후보 통계 ===
             장소  거래수      평균금액      최대금액
0   (주)더팬테스틱 브래    1   50400.0   50400.0
1   (주)라온사람들 서울    1  132000.0  132000.0
2   (주)새시대체인벨몽드    2  108460.0  187830.0
3     (주)신화케이푸드    1  157000.0  157000.0
4     (주)에스피씨삼립    1   87500.0   87500.0
5     (주)유포리막국수    5  109200.0  155000.0
6    (주)육설모 춘천점    1  133600.0  133600.0
7       (주)인터파크    1  105000.0  105000.0
8     (주)춘천민물장어    1  408010.0  408010.0
9      (주)텐시코리아    2   81500.0  112000.0
10  (주)호텔신라인천공항    1  527749.0  527749.0
11    (주)황솔촌상무점    1  180500.0  180500.0

평균금액 75th percentile (threshold): 162875.00

=== 임계값 이상으로 진짜 배달음식으로 의심되는 장소 ===
            장소  거래수      평균금액      최대금액
0    (주)춘천민물장어    1  408010.0  408010.0
1  (주)호텔신라인천공항    1  527749.0  527749.0
2    (주)황솔촌상무점    1  180500.0  180500.0


In [22]:
import pandas as pd

# 1. 엑셀 파일 경로
file_path = r"C:\bit_esg\python\data_anal\식사관련_업무추진비_정제본.xlsx"
df = pd.read_excel(file_path)

# 2. 장소 정제
df["장소"] = df["장소"].astype(str).str.strip()

# 3. 방문횟수 계산
visit_counts = df["장소"].value_counts().to_dict()
df["방문횟수"] = df["장소"].map(visit_counts)

# 4. 순위 계산: 동점자 동일 순위, 그 다음 순위는 건너뛰지 않음
rank_df = (
    df[["장소", "방문횟수"]]
    .drop_duplicates()
    .copy()
)
rank_df["순위"] = rank_df["방문횟수"].rank(method="dense", ascending=False).astype(int)

# 5. 병합 (장소별 방문횟수 및 순위)
final_df = df[["장소"]].drop_duplicates().copy()
final_df["방문횟수"] = final_df["장소"].map(visit_counts)
final_df = final_df.merge(rank_df[["장소", "순위"]], on="장소", how="left")

# 6. 저장
output_path = r"C:\bit_esg\python\data_anal\식사관련_장소_방문횟수_순위.xlsx"
final_df.to_excel(output_path, index=False)

print(f"✅ 저장 완료: {output_path}")


✅ 저장 완료: C:\bit_esg\python\data_anal\식사관련_장소_방문횟수_순위.xlsx


In [ ]:
# 3단계: 장소명 → 실제 주소 변환 (카카오 API 활용)

# 코드흐름
# 공공명칭에서 지리정보로 연결

import pandas as pd
import requests
import time
import urllib.parse

# 1. 카카오 REST API 키 (KakaoAK 포함)
KAKAO_API_KEY = "KakaoAK dbb446f02b5943b21fb8bf267c281757"

# 2. 엑셀 파일 로드
df = pd.read_excel("식사관련_장소별_빈도.xlsx")
df = df.dropna(subset=['장소'])
df['도로명주소'] = None
df['지번주소'] = None

# 3. 장소명 기반 주소 검색 함수 (춘천시 내에서만)
def get_address_by_place(place):
    base_url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": KAKAO_API_KEY}
    
    query = urllib.parse.quote(f"춘천 {place}")
    url = f"{base_url}?query={query}&size=3"

    res = requests.get(url, headers=headers)
    if res.status_code == 200:
        docs = res.json().get("documents", [])
        for doc in docs:
            address = doc.get("road_address_name") or doc.get("address_name", "")
            if address and "춘천" in address:
                return doc.get("road_address_name", ""), doc.get("address_name", "")
    return None, None

# 4. 주소 수집 반복
for i, row in df.iterrows():
    place = row['장소']
    road_addr, jibun_addr = get_address_by_place(place)
    df.at[i, '도로명주소'] = road_addr
    df.at[i, '지번주소'] = jibun_addr
    print(f"{place} → 도로명: {road_addr}, 지번: {jibun_addr}")
    time.sleep(0.4)  # 과호출 방지

# 5. 저장
df.to_excel("식당_주소_포함.xlsx", index=False)
print("✅ 주소 검색 및 저장 완료.")


In [ ]:
# 4단계: 주소 → 위도/경도 변환 + 지도 시각화

# 코드 흐름
# 위 장소를 지도에 마커로 시각적으로 표현

import pandas as pd
import folium
import requests
import urllib.parse
import time

# 카카오 API 키 (반드시 "KakaoAK " prefix 포함)
KAKAO_API_KEY = "KakaoAK dbb446f02b5943b21fb8bf267c281757"

# 1. 데이터 불러오기
df = pd.read_excel("식당_주소_포함.xlsx")
# 도로명주소 없는 행은 분석에서 제외
df = df.dropna(subset=['도로명주소'])

# 2. 위도, 경도 컬럼 추가
df['lat'] = None
df['lng'] = None

# 3. 카카오 주소 → 좌표 변환 함수
def get_coords_from_address(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": KAKAO_API_KEY}
    params = {"query": address}
    res = requests.get(url, headers=headers, params=params)
    if res.status_code == 200:
        docs = res.json().get("documents", [])
        if docs:
            # y: 위도, x: 경도
            return float(docs[0]['y']), float(docs[0]['x'])
    return None, None

# 4. 순회하며 좌표 저장
for i, row in df.iterrows():
    addr = row['도로명주소']
    lat, lng = get_coords_from_address(addr)
    df.at[i, 'lat'] = lat
    df.at[i, 'lng'] = lng
    print(f"{row['순위']}위 {row['장소']} → 위도: {lat}, 경도: {lng}")
    time.sleep(0.3)  # API 과다 호출 방지

# 5. 지도 생성 (춘천시 중심)
m = folium.Map(location=[37.8813, 127.7298], zoom_start=13)

# 6. (선택) 보기 편의를 위해 '건수' 내림차순 정렬
df = df.sort_values(by='건수', ascending=False).reset_index(drop=True)

# 7. 마커 추가 (엑셀의 '순위' 열을 아이콘 숫자로 표시)
for _, row in df.dropna(subset=['lat', 'lng']).iterrows():
    lat = row['lat']
    lng = row['lng']
    name = row['장소']
    count = row['건수']
    rank = int(row['순위'])   # 기존 엑셀에 있던 순위
    addr = row['도로명주소']

    popup_html = (
        f"<b>{name}</b><br>"
        f"순위: {rank}<br>"
        f"방문 횟수: {count}<br>"
        f"{addr}"
    )

    folium.map.Marker(
        location=[lat, lng],
        icon=folium.DivIcon(html=f"""
            <div style="
                font-size:12px;
                color:white;
                background-color:#2A81CB;
                border-radius:50%;
                text-align:center;
                width:24px;
                height:24px;
                line-height:24px;
            ">{rank}</div>
        """),
        tooltip=name,
        popup=folium.Popup(popup_html, max_width=300)
    ).add_to(m)

# 8. 결과 HTML로 저장
m.save("춘천_식당_순위지도.html")
print("✅ 순위 지도 생성 완료 → 춘천_식당_순위지도.html")


1위 나라앤미네 → 위도: 37.881154822492, 경도: 127.728581605956
2위 유일관 → 위도: 37.8802248054727, 경도: 127.729248871526
2위 유일관 → 위도: 37.8802248054727, 경도: 127.729248871526
3위 큰집한우 → 위도: 37.8778572619361, 경도: 127.734969692308
3위 큰집한우 → 위도: 37.8778572619361, 경도: 127.734969692308
4위 미다미 → 위도: 37.879671388192, 경도: 127.763741612937
4위 미다미 → 위도: 37.879671388192, 경도: 127.763741612937
5위 회영루 → 위도: 37.8808842082109, 경도: 127.726361519583
5위 회영루 → 위도: 37.8808842082109, 경도: 127.726361519583
6위 희정장어구이 → 위도: 37.8806791361099, 경도: 127.730528921438
6위 희정장어구이 → 위도: 37.8806791361099, 경도: 127.730528921438
7위 풍년소갈비살 → 위도: 37.876092642902, 경도: 127.724323825217
7위 풍년소갈비살 → 위도: 37.876092642902, 경도: 127.724323825217
8위 맥고을 → 위도: 37.8822876945977, 경도: 127.727584064863
8위 맥고을 → 위도: 37.8822876945977, 경도: 127.727584064863
9위 중화루 → 위도: 37.8755454305588, 경도: 127.735770180448
9위 중화루 → 위도: 37.8755454305588, 경도: 127.735770180448
10위 쟈스민 → 위도: 37.8764918017763, 경도: 127.731949123603
10위 쟈스민 → 위도: 37.8764918017763, 경도: 127.731949123603

In [ ]:
# 5단계: 장소명 → 대표 메뉴(소분류) 분류 + 카테고리 지도 시각화

import pandas as pd
import folium
import requests
import time

# 카카오 API 키 (반드시 "KakaoAK " prefix 포함)
KAKAO_API_KEY = "KakaoAK dbb446f02b5943b21fb8bf267c281757"

# 1. 데이터 불러오기
df = pd.read_excel("식당_주소_포함.xlsx")
df = df.dropna(subset=['도로명주소', '순위', '건수', '장소']).reset_index(drop=True)

# 2. 주소 → 좌표 변환 함수
def get_coords(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": KAKAO_API_KEY}
    params = {"query": address}
    res = requests.get(url, headers=headers, params=params)
    if res.status_code == 200:
        docs = res.json().get("documents", [])
        if docs:
            return float(docs[0]['y']), float(docs[0]['x'])
    return None, None

# 3. 장소명 → 대표메뉴(소분류) 추출 함수
def get_menu(place_name):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": KAKAO_API_KEY}
    # "춘천 {장소명}" 으로 검색
    params = {
        "query": f"춘천 {place_name}",
        "size": 1
    }
    res = requests.get(url, headers=headers, params=params)
    if res.status_code == 200:
        docs = res.json().get("documents", [])
        if docs:
            cat = docs[0].get("category_name", "")
            # "대분류 > 중분류 > 소분류" 형태, 마지막을 소분류로
            if " > " in cat:
                return cat.split(" > ")[-1]
            return cat
    return "기타"

# 4. 시트 순회하며 좌표 & 메뉴 채우기
df['lat'] = df['lng'] = None
df['대표메뉴'] = None

for i, row in df.iterrows():
    # 4-1. 좌표
    y, x = get_coords(row['도로명주소'])
    df.at[i, 'lat'] = y
    df.at[i, 'lng'] = x

    # 4-2. 대표메뉴
    menu = get_menu(row['장소'])
    df.at[i, '대표메뉴'] = menu

    print(f"{row['순위']}위 {row['장소']} → 메뉴: {menu}, 위치: ({y},{x})")
    time.sleep(0.3)

# 5. 색상 맵 & 고정 반경
unique_menus = df['대표메뉴'].unique()
colors = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#ffff33']
color_map = {m: colors[i % len(colors)] for i, m in enumerate(unique_menus)}
fixed_radius = 8

# 6. 지도 생성
m = folium.Map(location=[37.8813, 127.7298], zoom_start=13)

# 7. 메뉴별 레이어 그룹
groups = {menu: folium.FeatureGroup(name=menu).add_to(m)
          for menu in unique_menus}

# 8. 마커 추가
for _, r in df.dropna(subset=['lat','lng']).iterrows():
    grp = groups[r['대표메뉴']]
    popup = (
        f"<b>{r['장소']}</b><br>"
        f"순위: {int(r['순위'])}<br>"
        f"방문 횟수: {int(r['건수'])}<br>"
        f"인원수: {r.get('인원수', 'N/A')}명<br>"
        f"대표 메뉴: {r['대표메뉴']}"
    )
    folium.CircleMarker(
        location=[r['lat'], r['lng']],
        radius=fixed_radius,
        color=color_map[r['대표메뉴']],
        fill=True, fill_opacity=0.7,
        popup=folium.Popup(popup, max_width=300),
        tooltip=r['장소']
    ).add_to(grp)

# 9. 레이어 컨트롤 & 저장
folium.LayerControl(collapsed=False).add_to(m)
m.save("춘천_식당_메뉴기반지도.html")
print("✅ 완료: 춘천_식당_메뉴기반지도.html")


1위 나라앤미네 → 메뉴: 참치회, 위치: (37.881154822492,127.728581605956)
2위 유일관 → 메뉴: 불고기,두루치기, 위치: (37.8802248054727,127.729248871526)
3위 큰집한우 → 메뉴: 육류,고기, 위치: (37.8778572619361,127.734969692308)
4위 미다미 → 메뉴: 일식, 위치: (37.879671388192,127.763741612937)
5위 회영루 → 메뉴: 중국요리, 위치: (37.8808842082109,127.726361519583)
6위 희정장어구이 → 메뉴: 장어, 위치: (37.8806791361099,127.730528921438)
7위 풍년소갈비살 → 메뉴: 기타, 위치: (37.876092642902,127.724323825217)
8위 맥고을 → 메뉴: 한식, 위치: (37.8822876945977,127.727584064863)
9위 중화루 → 메뉴: 중국요리, 위치: (37.8755454305588,127.735770180448)
10위 쟈스민 → 메뉴: 피부관리, 위치: (37.8764918017763,127.731949123603)
11위 해신탕능이마을 → 메뉴: 한식, 위치: (37.8817359985888,127.71432408458)
12위 한어울 → 메뉴: 돈까스,우동, 위치: (37.8826904920201,127.730872295141)
13위 함지 → 메뉴: 양식, 위치: (37.8771525184483,127.723837048672)
14위 파도횟집 → 메뉴: 회, 위치: (37.8828196429404,127.730196779269)
15위 별당막국수 → 메뉴: 국수, 위치: (37.8720784028616,127.724288384555)
16위 점봉산산채 → 메뉴: 오리, 위치: (37.8654195239079,127.70583488057)
17위 해마 → 메뉴: 일식집, 위치: (37.887220419768,127.7510445